In [ ]:
import sys
sys.path.insert(0,"../..")
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from get_results import read_df, get_performance_profiles

# Circle Instance Results

In [ ]:
df_in = read_df("../data/random_circle.txt")
df_in.head()

In [ ]:
df_in["ratio"].unique()

There are some issues with the ratio, need to round $0.74\to 0.75$ and $0.24\to 0.25$

In [ ]:
df = df_in.copy()
df.loc[df["ratio"]==0.74,"ratio"]=0.75
df.loc[df["ratio"]==0.24,"ratio"]=0.25
df["ratio"].unique()

In [ ]:
df.head()

In [ ]:
savefig = False
fig_dir = "../../docs/paper/fig"

# Plot Style Guide

In [ ]:
CM = dict(zip(df['strategy'].unique(),[mpl.colormaps["Set1"](i) for i in range(len(df['strategy'].unique()))]))
LS = dict(zip(np.sort(df['ratio'].unique())[::-1],["-","--","-.",":","-","--"]))
MS = dict(zip(np.sort(df['ratio'].unique())[::-1],["o","^","D","x","",""]))
LS[0] = "-"
MS[0] = ""

# Confirm size

In [ ]:
tab = df[df["time_limit"]==600].pivot_table(index=["s","n","p"],columns=["solver","strategy","ratio"],values="solve_time",aggfunc="size")
tab

In [ ]:
_df = df[(df["time_limit"]==600)]
_df["solved"] = _df["total_time"] < 600
tab = _df.pivot_table(index=["s","n"],columns=["solver","strategy","ratio"],values="gap")
tab = tab[[
            (      'ct',         'na',  0.0),
            (  'glover',         'na',  0.0),
            ('coordpar',     'random', 0.25),
            ('coordpar',     'random',  0.5),
            ('coordpar',     'random', 0.75),
            ('coordpar', 'stratified', 0.25),
            ('coordpar', 'stratified',  0.5),
            ('coordpar', 'stratified', 0.75),
            ]]
tab *= 100
tab = tab.round(2)
tab = tab.style.format('{:.2f}')
print(tab.to_latex(multirow_align="naive",convert_css=True))
display(tab)
tab = _df.pivot_table(index=["s","n"],columns=["solver","strategy","ratio"],values="solved",aggfunc="sum")
tab = tab[[
            (      'ct',         'na',  0.0),
            (  'glover',         'na',  0.0),
            ('coordpar',     'random', 0.25),
            ('coordpar',     'random',  0.5),
            ('coordpar',     'random', 0.75),
            ('coordpar', 'stratified', 0.25),
            ('coordpar', 'stratified',  0.5),
            ('coordpar', 'stratified', 0.75),
            ]]
print(tab.style.to_latex(multirow_align="naive",convert_css=True))
display(tab)

In [ ]:
_df = df[(df["solver"]=="coordpar") ]# &(df["time_limit"] > 30)]
fig,axes = plt.subplots(2,figsize=(8,12),sharex=True,gridspec_kw={'height_ratios': [2, 1]})
for st in _df["strategy"].unique():
    for r in _df[_df["strategy"]==st]["ratio"].unique():
        G = _df[(_df["strategy"]==st) & (_df["ratio"]==r)].groupby(by=["time_limit"])["bound"].mean()
        axes[0].plot(G,color=CM[st],ls=LS[r],marker=MS[r],label=f"{st} {r}")
        G = _df[(_df["strategy"]==st) & (_df["ratio"]==r)].groupby(by=["time_limit"])["objval"].mean()
        axes[0].plot(G,color=CM[st],ls=LS[r],marker=MS[r])#,label=f"{st} {r}")
        G = _df[(_df["strategy"]==st) & (_df["ratio"]==r)].groupby(by=["time_limit"])["cuts"].mean()
        axes[1].plot(G,color=CM[st],ls=LS[r],marker=MS[r],label=f"{st} {r}")
axes[0].set_xlim(left=0)
# axes[0].set_ylim(bottom=0)
axes[0].set_xticks([30,60,120,300,600])
axes[0].set_ylabel("Primal-Dual Gap")
axes[1].set_ylabel("Average cuts added")
axes[0].legend()
axes[1].legend()
fig.supxlabel("Time (sec)")
fig.suptitle("Progress over time on ball instances")
plt.tight_layout()
if savefig: 
    plt.savefig(f"{fig_dir}/pp_ball.pdf")
plt.show()